En este nootbook trabajé con los datos de "crude oil" de la base de datos de la International Energy Statistics [database] (United Nations Statistics Division, 2018). Se puede obtener aqui: http://data.un.org/Explorer.aspx

Los datos a obtener eran el valor anual de la suma de las exportaciones menos las importaciones de petróleo:: crudo de los exportadores netos.
Asumismos que la base de datos se refiere a ellos como "Conventional Crude Oil"

In [1]:
import pandas as pd

In [2]:

#Leer la direccion, luego el archivo csv, luego transformalo en un DataFrame 
imports = "/Users/Deneb/git/Independent_Projects/PetroleoArellanes/Databases/UNSD/UNdata_Export_20200724_213546535.csv"
exports = "/Users/Deneb/git/Independent_Projects/PetroleoArellanes/Databases/UNSD/UNdata_Export_20200724_213715549.csv"

imports = pd.read_csv(imports)
exports = pd.read_csv(exports)

imports = pd.DataFrame(imports)
exports = pd.DataFrame(exports)

In [3]:
# #Obtener lista de exportadores del archivo proporcionadp
# exp_netos = pd.read_excel("../Exportadores netos 1990-2015.xlsx",  sheet_name="Lista 2", header=None)
# exp_netos = exp_netos[1].to_list()
# #Elimina los primeros tres, porque son NaN
# del exp_netos[0:3]

##Para conincidir con la base de datos traduje con google traslate los nombres de los paises
#exp_netos = ['Albania','Angola','Saudi Arabia','Algeria','Argentina','Benin','Azerbaijan','Bolivia','Brazil','Brunei','Cameroon','Canada','China','Colombia','Congo','Denmark','Ecuador','Egypt','UAE','Estonia','Gabon','Indonesia','Georgia','Ghana','Guatemala','Iran','Iraq','Kazakhstan','Kyrgyzstan','Kuwait','Libya','Malaysia','Mexico','Mongolia','Myanmar','Nigeria','Norway','Oman','Qatar','UK','RD of the Congo','Russia','Trinidad and Tobago','Sudan','South Sudan','Tunisia','Turkmenistan','Vietnam','Venezuela','Yemen']


##### Estos paises tenian nombres diferentes a los de la lista original
        # Bolivia = Bolivia (Plur. State of)
        # Brunei = Brunei Darussalam
        # UAE = United Arab Emirates
    # Estonia = no aparece en la base de datos ni como exportador ni como importador
        # Iran =  Iran (Islamic Rep. of)
        # UK = United Kingdom
        # RD of the Congo = Dem. Rep. of the Congo
        # Russia = Russian Federation 
        # Vietnam = Viet Nam
        # Venezuela = Venezuela (Bolivar. Rep.)


####Use este codigo para identificar cualese estaban y cuales no
# exporters_list = imports["Country or Area"].to_list()
# for x in netos:
#     if x not in exporters_list:
#         print(x)


##### Lista de productores netos adecuada a como aparecen los nombres en la base de datos
netos = ['Albania','Angola','Saudi Arabia','Algeria','Argentina','Benin','Azerbaijan','Bolivia (Plur. State of)','Brazil','Brunei Darussalam','Cameroon','Canada','China','Colombia','Congo','Denmark','Ecuador','Egypt','United Arab Emirates','Estonia','Gabon','Indonesia','Georgia','Ghana','Guatemala','Iran (Islamic Rep. of)','Iraq','Kazakhstan','Kyrgyzstan','Kuwait','Libya','Malaysia','Mexico','Mongolia','Myanmar','Nigeria','Norway','Oman','Qatar','United Kingdom','Dem. Rep. of the Congo','Russian Federation','Trinidad and Tobago','Sudan','South Sudan','Tunisia','Turkmenistan','Viet Nam','Venezuela (Bolivar. Rep.)','Yemen']

### Estos 18 paises no aparecen en la base de datos de importadores 
#['Angola', 'Saudi Arabia', 'Benin', 'Bolivia (Plur. State of)', 'Brunei Darussalam', 'Congo', 'United Arab Emirates', 'Estonia', 'Gabon', 'Iran (Islamic Rep. of)', 'Iraq', 'Kuwait', 'Libya', 'Mongolia', 'Nigeria', 'Oman', 'Qatar', 'South Sudan']

###Estos paises no aparecen en la base de datos de exportadores 
# ['Estonia']

En la lista de importadores hay: 32 paises con 658 registros 

En la lista de exportadores hay: 49 paises con 1240 registros

In [4]:
#Filtra en la base de datos (BD) solo los países que esten en la lista de productores netos
importers = imports.loc[imports["Country or Area"].isin(netos)]
exporters = exports.loc[exports["Country or Area"].isin(netos)]

#En ambas BD castea el valor de año, 
importers = importers.astype({"Year" : "int32"})
exporters = exporters.astype({"Year" : "int32"})

#En ambas BD elimina las columnas de unidad y transacción
importers = importers.drop(columns=["Commodity - Transaction", "Unit"])
exporters = exporters.drop(columns=["Commodity - Transaction", "Unit"])

#En ambas BD agrega la unidad al nombre de la columna, y cambia el nombre de las notas y pais
importers = importers.rename(columns={"Quantity Footnotes" : "imp_notes", "Quantity" : "Imports (Metric tons, thousand)", "Country or Area" : "Country"})
exporters = exporters.rename(columns={"Quantity Footnotes" : "exp_notes", "Quantity" : "Exports (Metric tons, thousand)", "Country or Area" : "Country"})

In [5]:
#Combina las dos BD en una sola
exp_imp = exporters.merge(importers, how="left", on=["Country", "Year"])  
#Transforma los valores Nan en la columna de importaciones en valores de 0
exp_imp["Imports (Metric tons, thousand)"] = exp_imp["Imports (Metric tons, thousand)"].fillna(0)
#Verifica que no se haya perdido ningun dato, comparando la extension de exportadores vs nueva BD
len(exporters) - len(exp_imp)

0

In [6]:
#Create a new column which substract imports to exports to get net exports
exp_imp["Net exports"] =  (exp_imp["Exports (Metric tons, thousand)"] - exp_imp["Imports (Metric tons, thousand)"])


In [32]:
netas = exp_imp
netas.head()

,Country,Year,"Exports (Metric tons, thousand)",exp_notes,"Imports (Metric tons, thousand)",imp_notes,Net exports
0,Albania,2017,488.00,NaN,0.0,NaN,488.00
1,Albania,2016,888.00,NaN,0.0,NaN,888.00
2,Albania,2015,987.00,NaN,0.0,NaN,987.00
3,Albania,2014,1007.22,NaN,0.0,NaN,1007.22
4,Albania,2013,1167.00,NaN,0.0,NaN,1167.00


# NEXT STEPS
- CREAR UNA TABLA EN QUE EL AÑO ESTE HORIZONTAL Y EL PAÍS EN VERTICAL / PARA ANALISIS VISUAL
- DEJAR LA OTRA TABLA PARA USO DE DATA 
- CREAR UNA GRAFICA

In [33]:
netas = netas.pivot_table("Net exports", ["Country"], "Year")
netas

Year,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Country,,,,,,,,,,,,,,,,,,,,,
Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,221.230000,586.47000,597.810000,920.070000,1167.00000,1007.220000,987.000000,888.00,488.000000,NaN
Algeria,14007.0,16182.0,15634.0,13944.0,15150.0000,14955.0000,17825.0000,17017.0000,19062.0000,19023.0000,...,34365.000000,32654.00000,32218.000000,31586.000000,27984.00000,23256.000000,23914.000000,24994.00,24455.000000,21676.000000
Angola,22000.0,23250.0,25000.0,23700.0,25500.0000,28065.0000,30794.0000,32723.0000,34386.0000,34448.0000,...,88622.000000,85997.00000,77278.000000,83390.000000,83470.00000,80395.000000,86919.000000,83953.00,74770.000000,NaN
Argentina,870.0,1053.0,3032.0,4306.0,9974.0000,13240.0000,15940.0000,16169.0000,15302.0000,12855.0000,...,4680.000000,2861.00000,3057.000000,2861.000000,1568.00000,1410.000000,1041.000000,1823.00,364.000000,NaN
Azerbaijan,NaN,NaN,NaN,NaN,-853.0000,-42.0000,0.0000,-28.0000,1812.0000,5684.0000,...,44331.000000,42659.00000,37268.000000,34728.000000,34260.00000,32903.000000,32843.000000,32479.00,30488.000000,NaN
Benin,193.0,185.0,128.0,210.0,177.0000,130.0000,112.0000,90.0000,49.0000,49.0000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bolivia (Plur. State of),0.0,49.0,17.0,63.0,67.0000,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.00000,0.000000,0.000000,353.00000,453.000000,59.000000,0.00,0.000000,NaN
Brazil,-28617.0,-26361.0,-26566.0,-25477.0,-27957.0000,-24676.0000,-28490.0000,-29077.0000,-27973.0000,-23711.0000,...,7640.000000,14846.00000,13829.000000,9556.000000,136.00000,8659.000000,22521.000000,35482.00,45922.000000,45634.000000
Brunei Darussalam,7333.0,8079.0,8800.0,8489.0,8684.0000,8369.0000,8030.0000,7920.0000,7136.0000,8373.0000,...,7064.000000,7342.00000,6740.000000,6920.000000,5822.00000,5362.000000,5763.000000,5209.00,4871.000000,4570.543000


In [44]:
totales = netas.fillna(0)
totales = netas.sum().map('{:.4f}'.format)
totales

Year
1990    1104018.0000
1991    1155210.0000
1992    1340739.0000
1993    1373438.0000
1994    1408232.6062
1995    1434760.2753
1996    1475248.6881
1997    1551026.4912
1998    1626918.4468
1999    1559618.9069
2000    1605441.7055
2001    1592267.7278
2002    1541323.6459
2003    1653686.2824
2004    1706719.5055
2005    1733527.7455
2006    1750652.8382
2007    1703736.0056
2008    1680766.7289
2009    1556973.2648
2010    1544169.3927
2011    1561657.6818
2012    1572659.6899
2013    1541548.7476
2014    1458901.9281
2015    1519536.2805
2016    1586573.7500
2017    1539094.9500
2018     756794.5088
dtype: object

Year
1990    float64
1991    float64
1992    float64
1993    float64
1994    float64
1995    float64
1996    float64
1997    float64
1998    float64
1999    float64
2000    float64
2001    float64
2002    float64
2003    float64
2004    float64
2005    float64
2006    float64
2007    float64
2008    float64
2009    float64
2010    float64
2011    float64
2012    float64
2013    float64
2014    float64
2015    float64
2016    float64
2017    float64
2018    float64
dtype: object

In [91]:
#netas.to_excel("../Excel_reports/Exportaciones_netas.xlsx")